Prepare database

In [ ]:
-- fill fact table
SET NOCOUNT ON
GO

DROP TABLE IF EXISTS [dbo].[Fact]
GO

CREATE TABLE [dbo].[Fact] (
	[Id] int identity(1,1) NOT NULL,
	[Dim1Id] int NOT NULL,
	[Dim2Id] int NOT NULL,
	[Dim3Id] int NOT NULL,
	[Dim4Id] int NOT NULL,
	[Dim5Id] int NOT NULL,
	[Dim6Id] int NOT NULL,
	[Dim7Id] int NOT NULL,
	[Dim8Id] int NOT NULL,
	[Dim9Id] int NOT NULL,
	[Dim10Id] int NOT NULL,
	[Measure1] decimal(36,2) NOT NULL,
	[Measure2] decimal(36,2) NOT NULL,
	[Measure3] decimal(36,2) NOT NULL,
	[Measure4] decimal(36,2) NOT NULL,
	[Measure5] decimal(36,2) NOT NULL,
	[Date1] date NOT NULL,
	[Date2] date NOT NULL,
	[Date3] date NOT NULL,
	[CreateDate] datetime NOT NULL DEFAULT GETDATE(),
	[ModifyDate] datetime NULL);
GO


-- Itzik Ben-Gan script to generate numbers table.
DROP TABLE IF EXISTS #NumbersTable;
GO

   WITH E00(N) AS (SELECT 1 UNION ALL SELECT 1),
        E02(N) AS (SELECT 1 FROM E00 a, E00 b),
        E04(N) AS (SELECT 1 FROM E02 a, E02 b),
        E08(N) AS (SELECT 1 FROM E04 a, E04 b),
        E16(N) AS (SELECT 1 FROM E08 a, E08 b),
        E32(N) AS (SELECT 1 FROM E16 a, E16 b),
   cteTally(N) AS (SELECT ROW_NUMBER() OVER (ORDER BY N) FROM E32)
 SELECT N
   INTO #NumbersTable
   FROM cteTally
  WHERE N <= 100000;
GO


-- Populate the fact basetable. 
DECLARE @Count int = 0;
WHILE (@Count < 25000000)
BEGIN
INSERT INTO [dbo].[Fact] WITH (HOLDLOCK TABLOCKX) 
		([Dim1Id] ,
	     [Dim2Id] ,
	     [Dim3Id] ,
	     [Dim4Id] ,
	     [Dim5Id] ,
	     [Dim6Id] ,
	     [Dim7Id] ,
	     [Dim8Id] ,
	     [Dim9Id] ,
	     [Dim10Id] ,
	     [Measure1] ,
	     [Measure2] ,
	     [Measure3] ,
	     [Measure4] ,
	     [Measure5] ,
	     [Date1] ,
	     [Date2] ,
	     [Date3] ) 
	SELECT	ABS(CHECKSUM(NEWID()) % 49) AS Dim1Id
               , ABS(CHECKSUM(NEWID()) % 49) AS Dim2Id
               , ABS(CHECKSUM(NEWID()) % 49) AS Dim3Id
               , ABS(CHECKSUM(NEWID()) % 49) AS Dim4Id
               , ABS(CHECKSUM(NEWID()) % 49) AS Dim5Id
               , ABS(CHECKSUM(NEWID()) % 49) AS Dim6Id
               , ABS(CHECKSUM(NEWID()) % 49) AS Dim7Id
               , ABS(CHECKSUM(NEWID()) % 49) AS Dim8Id
               , ABS(CHECKSUM(NEWID()) % 999) AS Dim9Id
               , ABS(CHECKSUM(NEWID()) % 999) AS Dim10Id
                
			,ROUND(RAND(CHECKSUM(NEWID())) * (10000), 1000) AS Measure1
			,ROUND(RAND(CHECKSUM(NEWID())) * (10000), 1000) AS Measure2
			,ROUND(RAND(CHECKSUM(NEWID())) * (10000), 1000) AS Measure3
			,ROUND(RAND(CHECKSUM(NEWID())) * (10000), 1000) AS Measure4
			,ROUND(RAND(CHECKSUM(NEWID())) * (10000), 1000) AS Measure5
			,DATEADD(DAY, RAND(CHECKSUM(NEWID()))*(1+DATEDIFF(DAY, '1/1/2016', '2/25/2020')), '1/1/2016') AS Date1
			,DATEADD(DAY, RAND(CHECKSUM(NEWID()))*(1+DATEDIFF(DAY, '1/1/2016', '2/25/2020')), '1/1/2016') AS Date2
			,DATEADD(DAY, RAND(CHECKSUM(NEWID()))*(1+DATEDIFF(DAY, '1/1/2016', '2/25/2020')), '1/1/2016') AS Date3
	FROM #NumbersTable
	SET @Count = @Count + @@ROWCOUNT;
END
GO


-- Clean up our numbers table.
DROP TABLE IF EXISTS #NumbersTable;
GO

Generate testdata

In [ ]:
-- Create partition functions + schema

CREATE PARTITION FUNCTION pf_PartitionFunction ( date )
AS RANGE RIGHT FOR VALUES ('20160101', '20170101', '20180101', '20190101', '20200101');
GO
CREATE PARTITION SCHEME ps_PartitionScheme
AS PARTITION pf_PartitionFunction
ALL TO ( [PRIMARY] );  
go



In [ ]:
-- create non-partitionned clustered columnstore table + fill

CREATE TABLE [dbo].[Fact_CS] (
	[Dim1Id] int NOT NULL,
	[Dim2Id] int NOT NULL,
	[Dim3Id] int NOT NULL,
	[Dim4Id] int NOT NULL,
	[Dim5Id] int NOT NULL,
	[Dim6Id] int NOT NULL,
	[Dim7Id] int NOT NULL,
	[Dim8Id] int NOT NULL,
	[Dim9Id] int NOT NULL,
	[Dim10Id] int NOT NULL,
	[Measure1] decimal(36,2) NOT NULL,
	[Measure2] decimal(36,2) NOT NULL,
	[Measure3] decimal(36,2) NOT NULL,
	[Measure4] decimal(36,2) NOT NULL,
	[Measure5] decimal(36,2) NOT NULL,
	[Date1] date NOT NULL,
	[Date2] date NOT NULL,
	[Date3] date NOT NULL,
	INDEX cci1 CLUSTERED COLUMNSTORE
) ;
GO

INSERT [dbo].[Fact_CS]
SELECT [Dim1Id],
	[Dim2Id] ,
	[Dim3Id] ,
	[Dim4Id] ,
	[Dim5Id] ,
	[Dim6Id] ,
	[Dim7Id] ,
	[Dim8Id] ,
	[Dim9Id] ,
	[Dim10Id] ,
	[Measure1] ,
	[Measure2] ,
	[Measure3] ,
	[Measure4] ,
	[Measure5] ,
	[Date1] ,
	[Date2] ,
	[Date3] 
	from [dbo].[Fact]
	order by [Date1]

In [ ]:
-- create partitionned clustered columnstore table + fill

CREATE TABLE [dbo].[Fact_PCS] (
	[Dim1Id] int NOT NULL,
	[Dim2Id] int NOT NULL,
	[Dim3Id] int NOT NULL,
	[Dim4Id] int NOT NULL,
	[Dim5Id] int NOT NULL,
	[Dim6Id] int NOT NULL,
	[Dim7Id] int NOT NULL,
	[Dim8Id] int NOT NULL,
	[Dim9Id] int NOT NULL,
	[Dim10Id] int NOT NULL,
	[Measure1] decimal(36,2) NOT NULL,
	[Measure2] decimal(36,2) NOT NULL,
	[Measure3] decimal(36,2) NOT NULL,
	[Measure4] decimal(36,2) NOT NULL,
	[Measure5] decimal(36,2) NOT NULL,
	[Date1] date NOT NULL,
	[Date2] date NOT NULL,
	[Date3] date NOT NULL,
	INDEX pcci1 CLUSTERED COLUMNSTORE
) ON ps_PartitionScheme(Date1);
GO

INSERT [dbo].[Fact_PCS]
SELECT [Dim1Id],
	[Dim2Id] ,
	[Dim3Id] ,
	[Dim4Id] ,
	[Dim5Id] ,
	[Dim6Id] ,
	[Dim7Id] ,
	[Dim8Id] ,
	[Dim9Id] ,
	[Dim10Id] ,
	[Measure1] ,
	[Measure2] ,
	[Measure3] ,
	[Measure4] ,
	[Measure5] ,
	[Date1] ,
	[Date2] ,
	[Date3] 
	from [dbo].[Fact]
	order by [Date1]

Check columnstore metadata

In [19]:
-- Check the deleted buffer.
SELECT  object_name(i.object_id) AS TableName,
		i.[name] AS IndexName, 
		p.[internal_object_type_desc] AS [Description],
		p.[rows] AS [RowCount], 
		p.[data_compression_desc] AS [CompressionType]
FROM [sys].[internal_partitions] AS p
INNER JOIN [sys].[indexes] AS i ON p.[object_id] = i.[object_id] 
	AND p.[index_id] = i.[index_id]
WHERE i.[name] like '%cci1'
ORDER BY object_name(i.object_id);
GO




-- Let's checkout our rowgroups.
SELECT  object_name(i.object_id) AS TableName,   
		i.name AS IndexName,   
		i.type_desc AS IndexType,   
		rg.state_desc AS StateDescription,
		rg.total_rows AS TotalRows,
		rg.deleted_rows AS DeletedRows,
		100*(ISNULL(deleted_rows,0))/NULLIF(total_rows,0) AS Fragmented,
		rg.trim_reason_desc AS TrimReason
FROM [sys].[indexes] AS i  
JOIN [sys].[dm_db_column_store_row_group_physical_stats] AS rg  
    ON i.object_id = rg.object_id AND i.index_id = rg.index_id
	WHERE i.name like '%cci1'
ORDER BY object_name(i.object_id);
GO

(7 rows affected)

(50 rows affected)

Total execution time: 00:00:00.079

TableName,IndexName,Description,RowCount,CompressionType
Fact_CS,cci1,COLUMN_STORE_DELETE_BITMAP,0,PAGE
Fact_PCS,pcci1,COLUMN_STORE_DELETE_BITMAP,0,PAGE
Fact_PCS,pcci1,COLUMN_STORE_DELETE_BITMAP,0,PAGE
Fact_PCS,pcci1,COLUMN_STORE_DELETE_BITMAP,0,PAGE
Fact_PCS,pcci1,COLUMN_STORE_DELETE_BITMAP,0,PAGE
Fact_PCS,pcci1,COLUMN_STORE_DELETE_BITMAP,0,PAGE
Fact_PCS,pcci1,COLUMN_STORE_DELETE_BITMAP,0,PAGE


TableName,IndexName,IndexType,StateDescription,TotalRows,DeletedRows,Fragmented,TrimReason
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,834176,0,0,RESIDUAL_ROW_GROUP
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,1048576,0,0,NO_TRIM
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,1048576,0,0,NO_TRIM
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,1048576,0,0,NO_TRIM
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,1048576,0,0,NO_TRIM
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,1048576,0,0,NO_TRIM
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,1048576,0,0,NO_TRIM
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,1048576,0,0,NO_TRIM
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,1048576,0,0,NO_TRIM
Fact_CS,cci1,CLUSTERED COLUMNSTORE,COMPRESSED,1048576,0,0,NO_TRIM


In [14]:
-- REORGANIZE
ALTER INDEX cci1 ON dbo.Fact_CS REORGANIZE;
GO
ALTER INDEX pcci1 ON dbo.Fact_PCS REORGANIZE;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:01:34.508

In [16]:
-- REORGANIZE COMPRESS_ALL_ROW_GROUPS
ALTER INDEX cci1 ON dbo.Fact_CS REORGANIZE
WITH (COMPRESS_ALL_ROW_GROUPS = ON);
GO
ALTER INDEX pcci1 ON dbo.Fact_PCS REORGANIZE
WITH (COMPRESS_ALL_ROW_GROUPS = ON);
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:21.405

In [18]:
-- REBUILD
ALTER INDEX cci1 ON dbo.Fact_CS REBUILD
WITH (MAXDOP = 1);
GO
ALTER INDEX pcci1 ON dbo.Fact_PCS REBUILD
WITH (MAXDOP = 1);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:18:25.139

Check compression

In [ ]:
SELECT OBJECT_NAME(object_id) AS [ObjectName], 
SUM(Rows) AS [RowCount], data_compression_desc AS [CompressionType]
FROM sys.partitions WITH (NOLOCK)
WHERE index_id < 2 --ignore the partitions from the non-clustered index if any
AND OBJECT_NAME(object_id) NOT LIKE N'sys%'
AND OBJECT_NAME(object_id) NOT LIKE N'queue_%' 
AND OBJECT_NAME(object_id) NOT LIKE N'filestream_tombstone%' 
AND OBJECT_NAME(object_id) NOT LIKE N'fulltext%'
AND OBJECT_NAME(object_id) NOT LIKE N'ifts_comp_fragment%'
AND OBJECT_NAME(object_id) NOT LIKE N'filetable_updates%'
AND OBJECT_NAME(object_id) NOT LIKE N'xml_index_nodes%'
AND OBJECT_NAME(object_id) NOT LIKE N'sqlagent_job%'  
AND OBJECT_NAME(object_id) NOT LIKE N'plan_persist%'  
GROUP BY object_id, data_compression_desc
ORDER BY SUM(Rows) DESC OPTION (RECOMPILE);
------
------


Check partitioning

In [ ]:
CREATE VIEW Partition_Info
AS
    SELECT  OBJECT_NAME(i.object_id) AS Object_Name ,
            i.index_id AS [Index_ID] ,
            p.partition_number ,
            fg.name AS Filegroup_Name ,
            p.rows ,
            au.total_pages ,
            f.name AS PartitionScheme ,
            CASE f.boundary_value_on_right
              WHEN 1 THEN 'less than'
              ELSE 'less than or equal to'
            END AS 'comparison' ,
            rv.value
    FROM    sys.partitions p
            JOIN sys.indexes i ON p.object_id = i.object_id
                                  AND p.index_id = i.index_id
            JOIN sys.partition_schemes ps ON ps.data_space_id = i.data_space_id
            JOIN sys.partition_functions f ON f.function_id = ps.function_id
            LEFT JOIN sys.partition_range_values rv ON f.function_id = rv.function_id
                                                       AND p.partition_number = rv.boundary_id
            JOIN sys.destination_data_spaces dds ON dds.partition_scheme_id = ps.data_space_id
                                                    AND dds.destination_id = p.partition_number
            JOIN sys.filegroups fg ON dds.data_space_id = fg.data_space_id
            JOIN ( SELECT   container_id ,
                            SUM(total_pages) AS total_pages
                   FROM     sys.allocation_units
                   GROUP BY container_id
                 ) AS au ON au.container_id = p.partition_id
    WHERE   i.index_id < 2 ;
     
GO

In [ ]:
SELECT  * FROM    Partition_Info ORDER BY 1 ,3

Data\_Modifications

In [ ]:
-- Let's checkout our rowgroups.
SELECT  object_name(i.object_id) AS TableName,   
		i.name AS IndexName,   
		i.type_desc AS IndexType,   
		rg.state_desc AS StateDescription,
		rg.total_rows AS TotalRows,
		rg.deleted_rows AS DeletedRows,
		rg.trim_reason_desc AS TrimReason
FROM [sys].[indexes] AS i  
JOIN [sys].[dm_db_column_store_row_group_physical_stats] AS rg  
    ON i.object_id = rg.object_id AND i.index_id = rg.index_id
	WHERE i.name like '%cci1';
GO



In [ ]:
-- Let's insert some data.
INSERT [dbo].[Fact_CS]
SELECT top 1000000 [Dim1Id],
	[Dim2Id] ,
	[Dim3Id] ,
	[Dim4Id] ,
	[Dim5Id] ,
	[Dim6Id] ,
	[Dim7Id] ,
	[Dim8Id] ,
	[Dim9Id] ,
	[Dim10Id] ,
	[Measure1] ,
	[Measure2] ,
	[Measure3] ,
	[Measure4] ,
	[Measure5] ,
	[Date1] ,
	[Date2] ,
	[Date3] 
	from [dbo].[Fact]
	order by [Date2]

INSERT [dbo].[Fact_PCS]
SELECT top 1000000 [Dim1Id],
	[Dim2Id] ,
	[Dim3Id] ,
	[Dim4Id] ,
	[Dim5Id] ,
	[Dim6Id] ,
	[Dim7Id] ,
	[Dim8Id] ,
	[Dim9Id] ,
	[Dim10Id] ,
	[Measure1] ,
	[Measure2] ,
	[Measure3] ,
	[Measure4] ,
	[Measure5] ,
	[Date1] ,
	[Date2] ,
	[Date3] 
	from [dbo].[Fact]
	order by [Date2]

In [ ]:
-- Let's checkout our rowgroups.
SELECT  object_name(i.object_id) AS TableName,   
		i.name AS IndexName,   
		i.type_desc AS IndexType,   
		rg.state_desc AS StateDescription,
		rg.total_rows AS TotalRows,
		rg.deleted_rows AS DeletedRows,
		rg.trim_reason_desc AS TrimReason
FROM [sys].[indexes] AS i  
JOIN [sys].[dm_db_column_store_row_group_physical_stats] AS rg  
    ON i.object_id = rg.object_id AND i.index_id = rg.index_id
	WHERE i.name like '%cci1';
GO


In [20]:
-- Now let's update some data.

UPDATE [dbo].[Fact_CS]
SET [Dim3Id] = 52
WHERE [Dim3Id] = 48;
GO
UPDATE [dbo].[Fact_PCS]
SET [Dim3Id] = 52
WHERE [Dim3Id] = 48;
GO

(0 rows affected)

(0 rows affected)

Total execution time: 00:00:02.908

In [ ]:
-- Let's checkout our rowgroups.
SELECT  object_name(i.object_id) AS TableName,   
		i.name AS IndexName,   
		i.type_desc AS IndexType,   
		rg.state_desc AS StateDescription,
		rg.total_rows AS TotalRows,
		rg.deleted_rows AS DeletedRows,
		rg.trim_reason_desc AS TrimReason
FROM [sys].[indexes] AS i  
JOIN [sys].[dm_db_column_store_row_group_physical_stats] AS rg  
    ON i.object_id = rg.object_id AND i.index_id = rg.index_id
	WHERE i.name like '%cci1';
GO

In [ ]:
-- Let's find out or min and max Id.
SELECT	MAX(Id) AS MaxId, 
		MIN(Id) AS MinId 
FROM Sales.SalesOrder_PCS;
GO




-- This will delete 1000 rows.
-- 500 will come from the delta store.
-- 500 will be marked as deleted
DELETE FROM Sales.SalesOrder_PCS
WHERE Id > 23800000;
GO

In [ ]:
-- Let's look at the deleted buffer.
SELECT  object_name(i.object_id) AS TableName,
		i.[name] AS IndexName, 
		p.[internal_object_type_desc] AS [Description],
		p.[rows] AS [RowCount], 
		p.[data_compression_desc] AS [CompressionType]
FROM [sys].[internal_partitions] AS p
	JOIN [sys].[indexes] AS i ON p.[object_id] = i.[object_id] 
	AND p.[index_id] = i.[index_id]
WHERE i.[name] like '%cci1';
GO

Check

In [ ]:
-- Let's look at the deleted buffer.
SELECT  object_name(i.object_id) AS TableName,
		i.[name] AS IndexName, 
		p.[internal_object_type_desc] AS [Description],
		p.[rows] AS [RowCount], 
		p.[data_compression_desc] AS [CompressionType]
FROM [sys].[internal_partitions] AS p
INNER JOIN [sys].[indexes] AS i ON p.[object_id] = i.[object_id] 
	AND p.[index_id] = i.[index_id]
WHERE i.[name] = 'pcci1';
GO

-- Let's checkout our rowgroups.
SELECT  object_name(i.object_id) AS TableName,   
		i.name AS IndexName,   
		i.type_desc AS IndexType,   
		rg.state_desc AS StateDescription,
		rg.total_rows AS TotalRows,
		rg.deleted_rows AS DeletedRows,
		100*(ISNULL(deleted_rows,0))/NULLIF(total_rows,0) AS Fragmented,
		rg.trim_reason_desc AS TrimReason
FROM [sys].[indexes] AS i  
JOIN [sys].[dm_db_column_store_row_group_physical_stats] AS rg  
    ON i.object_id = rg.object_id AND i.index_id = rg.index_id
	WHERE i.name = 'pcci1';
GO